## Chapter 3 - Item-based Collaborative Filtering

Before we start, we have to make sure that we already have our dependencies, that is 'recsys' folder

In [1]:
import os
#Check if we already have the 'recsys' folder
if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    # If not then download directly from the source
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip    
    !unzip recsys.zip

--2023-01-03 20:58:12--  https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip [following]
--2023-01-03 20:58:12--  https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15312323 (15M) [application/zip]
Saving to: ‘recsys.zip’

recsys.zip          100%[===================>]  14.60M  --.-KB/s    in 0.05s   

2023-01-03 20:58:12 (317 MB/s) - ‘recsys.zip’ saved [

### Requirements
Other than the 'recsys' folder, we also have to make sure that the other required libs have already been installed<br>
```
matplotlib==3.2.2
numpy==1.19.2
pandas==1.0.5
python==3.7
scikit-learn==0.24.1
scikit-surprise==1.1.1
scipy==1.6.2
```
(If we use Google Colab, most of these libs are already installed and up-to-date, except for *scikit-surprise* which is not pre-installed by Google Colab) 

To install scikit-surprise on Google Colab, we must execute the code below
```
!pip install surprise
```
But this notebook doesn't require this library yet, so we're not going to install scikit-surprise at the moment

Import all of the required libs

In [2]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

from recsys.datasets import ml1m, ml100k
from recsys.preprocessing import ids_encoder

import pandas as pd
import numpy as np
import os
import sys

### Load Movielens Data

Instead of using Movielens Latest Small Data, we're going to use Movielens 100K Data as it is a stable benchmark and won't change over time

In [3]:
ratings, movies = ml100k.load()

Download data 100.2%
Successfully downloaded ml-100k.zip 4924029 bytes.
Unzipping the ml-100k.zip zip file ...


What's the difference between *Movielens Latest Small Data* and *Movielens 100K Data*?<br>
According to <a href="https://grouplens.org/datasets/movielens/">GroupLens</a>, *Movielens Latest Small Data* will change overtime while *Movielens 100K Data* is a stable benchmark released in 1998 and won't change over time

What's the difference between those two data in terms of content?

In [4]:
ratings.head()

,userid,itemid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


*Movielens Latest Small Data*'s $ratings$ has a timestamp column while *Movielens 100K Data*'s $ratings$ doesn't have that column 

In [5]:
movies.head()

,itemid,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


*Movielens Latest Small Data*'s $movies$ has a genres column while *Movielens 100K Data*'s $movies$ doesn't have that column 

### Encoding of *userids* and *itemids*

All *userid* and *itemid* in $ratings$ could have a non-consecutive sequence when ids are ordered, <br>
For convenience at the construction of the matrix, encode each of those to a consecutive sequence when ids are ordered through LabelEncoder

In [6]:
# Encode userids and itemids in ratings through LabelEncoder
# uencoder -> LabelEncoder object of userids
# iencoder -> LabelEncoder object of itemids
ratings, uencoder, iencoder = ids_encoder(ratings)

Now that the initial configuration is done, let's implement the item-based collaborative filtering algorithm

Item-based collaborative filtering is still a part of memory-based collaborative filtering. <br>
The user-based collaborative filtering has already been explained in the previous chapter, so now we'll take a look at the item-based collaborative filtering

# 2. Item-to-Item Collaborative Filtering

## Idea
Let $u$ be the active user and $i$ the referenced item
1. If $u$ liked items similar to $i$, he will probably like item $i$.
2. If he hated or disliked items similar to $i$, he will also hate item $i$.

The idea is therefore to look at how an active user $u$ rated items similar to $i$ to know how he would have rated item $i$

## Algorithm : item-to-item collaborative filtering

The algorithm that defines item-based CF is described as follow <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a><a href="https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.554.1671&rep=rep1&type=pdf">(George Karypis 2001)</a> :

<ol>
    <li>
        First identify the $k$ most similar items for each item in the catalogue and record the corresponding similarities. To compute similarity between two items we can user the <i>Adjusted Cosine Similarity</i> that has proven to be more efficient than the basic <i>Cosine similarity measure</i> used for user-based collaborative as described in <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a>. The Adjusted Cosine distance between two items $i$ and $j$ is computed as follow

\begin{equation}
 w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}
\end{equation}

$w_{i,j}$ is the degree of similarity between items $i$ and $j$. This term is computed for all users $u\in U$, where $U$ is the set of users that rated both items $i$ and $j$. Let's denote by $S^{(i)}$ the set of the $k$ most similar items to item $i$.
    </li>    
    <li> To produce top-N recommendations for a given user $u$ that has already purchased a set $I_u$ of items, do the following :
<ul>
    <li> Find the set $C$ of candidate items by taking the union of all $S^{(i)}, \forall i\in I_u$ and removing each of the items in the set $I_u$.
\begin{equation}
 C = \bigcup_{i\in I_u}\{S^{(i)}\}\smallsetminus I_u
\end{equation}
    </li>
    <li>
        $\forall c\in C$, compute similarity between c and the set $I_u$ as follows:
\begin{equation}
 w_{c,I_u} = \sum_{i\in I_u} w_{c,i}, \forall c \in C
\end{equation}
    </li>
    <li>
        Sort items in $C$ in decreasing order of $w_{c,I_u}, \forall c \in C$, and return the first $N$ items as the Top-N recommendation list.
    </li>
</ul>    
    </li>
</ol>

Before returning the first $N$ items as top-N recommendation list, we can make predictions about what user $u$ would have given to each items in the top-N recommendation list, rearrange the list in descending order of predicted ratings and return the rearranged list as the final recommendation list. Rating prediction for item-based CF is given by the following formular <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a>:

\begin{equation}
 \hat{r}_{u,i}=\frac{\sum_{i\in S^{(i)}}r_{u,j}\cdot w_{i,j}}{\sum_{j\in S^{(i)}}|w_{i,j}|}
\end{equation}

### Step 1. Find similarities for each of the items

To compute similarity between two items $i$ and $j$, we need to :

1. Find all users who rated both of them,
2. Normalize their ratings on items $i$ and $j$
3. Apply the cosine metric to the normalized ratings to compute similarity between $i$ and $j$

Function ```normalize()``` process the rating dataframe to normalize ratings of all users

In [7]:
def normalize():
    # compute mean rating for each user
    mean = ratings.groupby(by='userid', as_index=False)['rating'].mean()
    norm_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userid')
    
    # normalize each rating by substracting the mean rating of the corresponding user
    norm_ratings['norm_rating'] = norm_ratings['rating'] - norm_ratings['rating_mean']
    return mean.to_numpy()[:, 1], norm_ratings

In [8]:
mean, norm_ratings = normalize()
np_ratings = norm_ratings.to_numpy()
norm_ratings.head()

,userid,itemid,rating,rating_mean,norm_rating
0,0,0,5,3.610294,1.389706
1,0,1,3,3.610294,-0.610294
2,0,2,4,3.610294,0.389706
3,0,3,3,3.610294,-0.610294
4,0,4,3,3.610294,-0.610294


Now that each rating has been normalized, we can represent each item by a vector of its normalized ratings

In [9]:
def item_representation(ratings):    
    return csr_matrix(
        pd.crosstab(ratings.itemid, ratings.userid, ratings.norm_rating, aggfunc=sum).fillna(0).values
    )

In [10]:
R = item_representation(norm_ratings)

In [11]:
display(R)
print('Total users: {}'.format(len(ratings['userid'].unique())))
print('Total movies: {}'.format(len(ratings['itemid'].unique())))

<1682x943 sparse matrix of type '<class 'numpy.float64'>'
	with 99650 stored elements in Compressed Sparse Row format>

Total users: 943
Total movies: 1682


1682 rows represents the movies through movieid, while 943 columns represents the users through userid 

Let's build and fit our $k$-NN model using sklearn

In [12]:
def create_model(rating_matrix, k=20, metric="cosine"):
    """
    :param R : numpy array of item representations
    :param k : number of nearest neighbors to return    
    :return model : our knn model
    """    
    model = NearestNeighbors(metric=metric, n_neighbors=k+1, algorithm='brute')
    model.fit(rating_matrix)    
    return model

#### Similarities computation

Similarities between items can be measured with the *Cosine* or *Eucliedian* distance. The ***NearestNeighbors*** class from the sklearn library simplifies the computation of neighbors. We just need to specify the metric (e.g. cosine or euclidian) that will be used to compute similarities.

The above method, ```create_model```, creates the kNN model and the following ```nearest_neighbors``` method uses the created model to kNN items. It returns nearest neighbors as well as similarities measures for each items.

```nearest_neighbors``` returns :
- ```similarities``` : numpy array of shape $(n,k)$
- ```neighbors``` : numpy array of shape $(n,k)$

where $n$ is the total number of items and $k$ is the number of neighbors to return, specified when creating the kNN model.

In [13]:
def nearest_neighbors(rating_matrix, model):
    """
    compute the top n similar items for each item.    
    :param rating_matrix : items representations
    :param model : nearest neighbors model    
    :return similarities, neighbors
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)    
    return similarities[:,1:], neighbors[:,1:]

#### Adjusted Cosine Similarity
In the context of item-based collaborative filtering, the adjusted cosine similarity has shown to be more efficient that the cosine or the euclidian distance. Here is the formula to compute the adjusted cosine weight between two items $i$ and $j$ :

\begin{equation}
 w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}.
\end{equation}

This term is computed for all users $u\in U$, where $U$ is the set of users that rated both items $i$ and $j$. Since the *sklearn* library do not directly implement the adjusted cosine similarity metric, we will implement it with the method ```adjusted_cosine```, with some helper function :

- ```save_similarities``` : since the computation of the adjusted cosine similarity is time consuming, around 5 mins for the ml100k dataset, we use this method to save the computed similarities for lated usage.
- ```load_similarities``` : load the saved similarities
- ```cosine``` : cosine distance between two vectors.

In [14]:
def save_similarities(similarities, neighbors, dataset_name):    
    base_dir = 'recsys/weights/item2item'
    save_dir = os.path.join(base_dir, dataset_name)
    os.makedirs(save_dir, exist_ok=True)    
    similarities_file_name = os.path.join(save_dir, 'similarities.npy')
    neighbors_file_name = os.path.join(save_dir, 'neighbors.npy')    
    try:
        np.save(similarities_file_name, similarities)
        np.save(neighbors_file_name, neighbors)        
    except ValueError as error:
        print(f"An error occured when saving similarities, due to : \n ValueError : {error}")

        
def load_similarities(dataset_name, k=20):
    base_dir = 'recsys/weights/item2item'
    save_dir = os.path.join(base_dir, dataset_name)    
    similiraties_file = os.path.join(save_dir, 'similarities.npy')
    neighbors_file = os.path.join(save_dir, 'neighbors.npy')    
    similarities = np.load(similiraties_file)
    neighbors = np.load(neighbors_file)    
    return similarities[:,:k], neighbors[:,:k]


def cosine(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))


def adjusted_cosine(np_ratings, nb_items, dataset_name):
    similarities = np.zeros(shape=(nb_items, nb_items))
    similarities.fill(-1)
    
    def _progress(count):
        sys.stdout.write('\rComputing similarities. Progress status : %.1f%%' % (float(count / nb_items)*100.0))
        sys.stdout.flush()
        
    items = sorted(ratings.itemid.unique())    
    for i in items[:-1]:
        for j in items[i+1:]:            
            scores = np_ratings[(np_ratings[:, 1] == i) | (np_ratings[:, 1] == j), :]
            vals, count = np.unique(scores[:,0], return_counts = True)
            scores = scores[np.isin(scores[:,0], vals[count > 1]),:]

            if scores.shape[0] > 2:
                x = scores[scores[:, 1].astype('int') == i, 4]
                y = scores[scores[:, 1].astype('int') == j, 4]
                w = cosine(x, y)

                similarities[i, j] = w
                similarities[j, i] = w
        _progress(i)
    _progress(nb_items)
    
    # get neighbors by their neighbors in decreasing order of similarities
    neighbors = np.flip(np.argsort(similarities), axis=1)
    
    # sort similarities in decreasing order
    similarities = np.flip(np.sort(similarities), axis=1)
    
    # save similarities to disk
    save_similarities(similarities, neighbors, dataset_name=dataset_name) 
    
    return similarities, neighbors

Now, we can call the ```adjusted_cosine``` function to compute and save items similarities and neighbors based on the adjusted cosine metric. 

Run these lines to compute the adjusted cosine between all items.
```
nb_items = ratings.itemid.nunique()
similarities, neighbors = adjusted_cosine(np_ratings, nb_items=nb_items, dataset_name='ml100k')
```
As the precomputed version is already available in *recsys*, we don't have to run those lines again. Just load the precomputed version instead for further use.

Among the following similarity metrics, choose the one you wish to use for the item-based collaborative filtering :

- **euclidian** or **cosine** : choose *euclidian* or *cosine* to initialize the similarity model through the sklearn library.
- **adjusted_cosine** : choose the *adjusted_cosine* metric to load similarities computed and saved through the ```adjusted_cosine``` function.

In this case, we will use the *adjusted_cosine* metric.

In [15]:
metric = 'adjusted_cosine' # [cosine, euclidean, adjusted_cosine]

if metric == 'adjusted_cosine':
    similarities, neighbors = load_similarities('ml100k')
else:
    model = create_model(R, k=21, metric=metric)
    similarities, neighbors = nearest_neighbors(R, model)

In [16]:
print('neighbors shape : ', neighbors.shape)
print('similarities shape : ', similarities.shape)

neighbors shape :  (1682, 20)
similarities shape :  (1682, 20)


```neighbors``` and ```similarities``` are numpy array, were each entries are list of 20 neighbors with their corresponding similarities

### Step 2. Top-N recommendation for a given user


For a user $u$ who has already rated a set of items $I_u$, give top-N movies recommendation

#### 2.1. Finding candidate items

To find candidate items for user $u$, we need to :

1. Find the set $I_u$ of items already rated by user $u$,
2. Take the union of similar items as $C$ for all items in $I_u$
3. Exclude from the set $C$ all items in $I_u$, to avoid recommend to a user items he has already purchased.

These are done in function ```candidate_items()```

In [17]:
def candidate_items(userid):
    """
    :param userid : user id for which we wish to find candidate items    
    :return : I_u, candidates
    """
    
    # 1. Finding the set I_u of items already rated by user userid
    I_u = np_ratings[np_ratings[:, 0] == userid]
    I_u = I_u[:, 1].astype('int')
    
    # 2. Taking the union of similar items for all items in I_u to form the set of candidate items
    c = set()
        
    for iid in I_u:    
        # add the neighbors of item iid in the set of candidate items
        c.update(neighbors[iid])
        
    c = list(c)
    # 3. exclude from the set C all items in I_u.
    candidates = np.setdiff1d(c, I_u, assume_unique=True)
    
    return I_u, candidates

In [18]:
#Take a user of testing later
test_user = uencoder.transform([1])[0]
#Find items purchased and candidates items for that user
i_u, u_candidates = candidate_items(test_user)

In [19]:
print('number of items purchased by user 1 : ', len(i_u))
print('number of candidate items for user 1 : ', len(u_candidates))

number of items purchased by user 1 :  272
number of candidate items for user 1 :  893


#### 2.2. Find similarity between each candidate item and the set $I_u$

In [20]:
def similarity_with_Iu(c, I_u):
    """
    compute similarity between an item c and a set of items I_u. For each item i in I_u, get similarity between 
    i and c, if c exists in the set of items similar to itemid.    
    :param c : itemid of a candidate item
    :param I_u : set of items already purchased by a given user    
    :return w : similarity between c and I_u
    """
    w = 0    
    for iid in I_u :        
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        if c in neighbors[iid] :
            w = w + similarities[iid, neighbors[iid] == c][0]    
    return w

#### 2.3. Rank candidate items according to their similarities to $I_u$

In [21]:
def rank_candidates(candidates, I_u):
    """
    rank candidate items according to their similarities with i_u    
    :param candidates : list of candidate items
    :param I_u : list of items purchased by the user    
    :return ranked_candidates : dataframe of candidate items, ranked in descending order of similarities with I_u
    """
    
    # list of candidate items mapped to their corresponding similarities to I_u
    sims = [similarity_with_Iu(c, I_u) for c in candidates]
    candidates = iencoder.inverse_transform(candidates)    
    mapping = list(zip(candidates, sims))
    
    ranked_candidates = sorted(mapping, key=lambda couple:couple[1], reverse=True)    
    return ranked_candidates

####2.4. Putting all together

Now that we defined all functions necessary to build our item to item top-N recommendation, let's define function ```topn_recommendation()``` that makes top-$N$ recommendations for a given user 

In [22]:
def topn_recommendation(userid, N=30):
    """
    Produce top-N recommendation for a given user    
    :param userid : user for which we produce top-N recommendation
    :param n : length of the top-N recommendation list    
    :return topn
    """
    # find candidate items
    I_u, candidates = candidate_items(userid)
    
    # rank candidate items according to their similarities with I_u
    ranked_candidates = rank_candidates(candidates, I_u)
    
    # get the first N row of ranked_candidates to build the top N recommendation list
    topn = pd.DataFrame(ranked_candidates[:N], columns=['itemid','similarity_with_Iu'])    
    topn = pd.merge(topn, movies, on='itemid', how='inner')    
    return topn

In [23]:
#Show top-30 movies recommendation for the test user
topn_recommendation(test_user)

,itemid,similarity_with_Iu,title
0,1356,52.867173,Ed's Next Move (1996)
1,1189,50.362199,Prefontaine (1997)
2,1516,31.133267,"Wedding Gift, The (1994)"
3,1550,31.031738,Destiny Turns on the Radio (1995)
4,1554,27.364494,Safe Passage (1994)
5,1600,27.287712,Guantanamera (1994)
6,1223,26.631850,King of the Hill (1993)
7,1388,26.624397,Gabbeh (1996)
8,766,26.590175,Man of the Year (1995)
9,691,26.461802,Dark City (1998)


This dataframe represents the top N recommendation list a user. These items are sorted in decreasing order of similarities with $I_u$.

**Observation** : The recommended items are the most similar to the set $I_u$ of items already purchased by the user.

### Extra: Top-N recommendation with rating *predictions*

Before recommending the previous list to the user, we can go further and predict the ratings the user would have given to each of these items, sort them in descending order of prediction and return the reordered list as the new top N recommendation list.

#### Rating prediction

As stated earlier, the predicted rating $\hat{r}_{u,i}$ for a given user $u$ on an item $i$ is obtained by aggregating ratings given by $u$ on items similar to $i$ as follows:

\begin{equation}
 \hat{r}_{u,i}=\frac{\sum_{j\in S^{(i)}}r_{u,j}\cdot w_{i,j}}{\sum_{j\in S^{(i)}}|w_{i,j}|}
\end{equation}

In [24]:
def predict(userid, itemid):
    """
    Make rating prediction for user userid on item itemid    
    :param userid : id of the active user
    :param itemid : id of the item for which we are making prediction        
    :return r_hat : predicted rating
    """
    
    # Get items similar to item itemid with their corresponding similarities
    item_neighbors = neighbors[itemid]
    item_similarities = similarities[itemid]
    
    # get ratings of user with id userid
    uratings = np_ratings[np_ratings[:, 0].astype('int') == userid]
    
    # similar items rated by item the user of i
    siru = uratings[np.isin(uratings[:, 1], item_neighbors)]
    scores = siru[:, 2]
    indexes = [np.where(item_neighbors == iid)[0][0] for iid in siru[:,1].astype('int')]    
    sims = item_similarities[indexes]
    
    dot = np.dot(scores, sims)
    som = np.sum(np.abs(sims))

    if dot == 0 or som == 0:
        return mean[userid]
    
    return dot / som

Now let's use our ```predict()``` function to predict what ratings the user would have given to the previous top-$N$ list and return the reorganised list (in decreasing order of predictions) as the new top-$N$ list

In [25]:
def topn_prediction(userid):
    """
    :param userid : id of the active user    
    :return topn : initial topN recommendations returned by the function topn_recommendation
    :return topn_predict : topN recommendations reordered according to rating predictions
    """
    # make top N recommendation for the active user
    topn = topn_recommendation(userid)
    
    # get list of items of the top N list
    itemids = topn.itemid.to_list()
    
    predictions = []
    
    # make prediction for each item in the top N list
    for itemid in itemids:
        r = predict(userid, itemid)
        
        predictions.append((itemid,r))
    
    predictions = pd.DataFrame(predictions, columns=['itemid','prediction'])
    
    # merge the predictions to topN_list and rearrange the list according to predictions
    topn_predict = pd.merge(topn, predictions, on='itemid', how='inner')
    topn_predict = topn_predict.sort_values(by=['prediction'], ascending=False)
    
    return topn, topn_predict

Now, let's make recommendation for test user and compare both list

In [26]:
topn, topn_predict = topn_prediction(userid=test_user)
# topn -> first list
# topn_predict -> second list

In [27]:
topn_predict

,itemid,similarity_with_Iu,title,prediction
7,1388,26.624397,Gabbeh (1996),4.666667
18,359,22.973658,"Assignment, The (1997)",4.600000
4,1554,27.364494,Safe Passage (1994),4.500000
14,1538,24.492453,All Over Me (1997),4.500000
27,1448,20.846909,My Favorite Season (1993),4.490052
29,1375,20.627152,"Cement Garden, The (1993)",4.333333
26,1466,21.063269,Margaret's Museum (1995),4.271915
2,1516,31.133267,"Wedding Gift, The (1994)",4.000000
23,1467,21.861203,"Saint of Fort Washington, The (1993)",4.000000
21,1537,22.061914,Cosi (1996),4.000000


As you will have noticed, the two lists are sorted in different ways. The second list is organized according to the predictions made for the user.

<b>Note</b>: When making predictions for user $u$ on item $i$, user $u$ may not have rated any of the $k$ most similar items to i. In this case, we consider the mean rating of $u$ as the predicted value.

#### Evaluation with Mean Absolute Error

In [28]:
from recsys.preprocessing import train_test_split, get_examples

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column='rating')

# train test split
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

def evaluate(x_test, y_test):
    print('Evaluate the model on {} test data ...'.format(x_test.shape[0]))
    preds = list(predict(u,i) for (u,i) in x_test)
    mae = np.sum(np.absolute(y_test - np.array(preds))) / x_test.shape[0]
    print('\nMAE :', mae)
    return mae

In [29]:
evaluate(x_test, y_test)

Evaluate the model on 10000 test data ...

MAE : 0.672389703640273


0.672389703640273

According to the result, the MAE is < 1.0, which is quite okay

### Summary

For Convenience, Item-based Collaborative Filtering can be used by calling ItemToItem class from *recsys*<br>
On how to use that class, the process is described as below

##### Evaluation on the ML-100K dataset

In [30]:
from recsys.memories.ItemToItem import ItemToItem
from recsys.preprocessing import ids_encoder, train_test_split, get_examples
from recsys.datasets import ml100k

# load data
ratings, movies = ml100k.load()

# prepare data
ratings, uencoder, iencoder = ids_encoder(ratings)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column='rating')

# train test split
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

Instantiate the ItemToItem Collaborative Filtering

Parameters :
- ```k``` : number of neighbors to consider for each item
- ```metric``` : metric to use when computing similarities : let's use **cosine**
- ```dataset_name``` : in this example, we use the ml100k dataset

In [31]:
# create the Item-based CF
item2item = ItemToItem(ratings, movies, k=20, metric='cosine', dataset_name='ml100k')

Normalize ratings ...
Create the similarity model ...
Compute nearest neighbors ...
Item to item recommendation model created with success ...


In [32]:
# evaluate the algorithm on test dataset
item2item.evaluate(x_test, y_test)

Evaluate the model on 10000 test data ...

MAE : 0.507794195659005


0.507794195659005

##### Evaluation on the ML-1M dataset

In [33]:
from recsys.memories.ItemToItem import ItemToItem
from recsys.preprocessing import ids_encoder, train_test_split, get_examples
from recsys.datasets import ml1m

# load data
ratings, movies = ml1m.load()

# prepare data
ratings, uencoder, iencoder = ids_encoder(ratings)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column='rating')

# train test split
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

# create the Item-based CF
item2item = ItemToItem(ratings, movies, k=20, metric='cosine', dataset_name='ml1m')

# evaluate the algorithm on test dataset
print("=========================")
item2item.evaluate(x_test, y_test)

Download data 100.1%
Successfully downloaded ml-1m.zip 5917549 bytes.
Unzipping the ml-1m.zip zip file ...
Normalize ratings ...
Create the similarity model ...
Compute nearest neighbors ...
Item to item recommendation model created with success ...
Evaluate the model on 100021 test data ...

MAE : 0.42514728655396045


0.42514728655396045

## Advantages over user-based CF

1. <b> Stability </b> : Items ratings are more stable than users ratings. New ratings on items are unlikely to significantly change the similarity between two items, particularly when the items have many ratings <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>. 
2. <b> Scalability </b> : with stable item's ratings, it is reasonable to pre-compute similarities between items in an item-item similarity matrix (similarity between items can be computed offline). This will reduce the scalability concern of the algorithm. <a href="https://dl.acm.org/doi/10.1145/371920.372071">(Sarwar <i>et al.</i> 2001)</a>, <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>.

## References

1. George Karypis (2001)<a href="https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.554.1671&rep=rep1&type=pdf">Evaluation of Item-Based Top-N Recommendation Algorithms</a>
2. Sarwar et al. (2001) <a href="https://dl.acm.org/doi/10.1145/371920.372071"> Item-based collaborative filtering recommendation algorithms</a> 
3. Michael D. Ekstrand, et al. (2011). <a href="https://dl.acm.org/doi/10.1561/1100000009"> Collaborative Filtering Recommender Systems</a>
4. J. Bobadilla et al. (2013)<a href="https://romisatriawahono.net/lecture/rm/survey/information%20retrieval/Bobadilla%20-%20Recommender%20Systems%20-%202013.pdf"> Recommender systems survey</a>
5. Greg Linden, Brent Smith, and Jeremy York (2003) <a href="https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf">Amazon.com Recommendations : Item-to-Item Collaborative Filtering</a>

## Author

[Carmel WENGA](https://www.linkedin.com/in/carmel-wenga-871876178/), <br>
PhD student at Université de la Polynésie Française, <br> 
Applied Machine Learning Research Engineer, <br>
[ShoppingList](https://shoppinglist.cm), NzhinuSoft.